# OBTENCION DE DATOS
**IMPORTAR LIBRERIAS Y OBTENER EL DATASET**

En primer lugar, importaremos las siguientes librerias necesarias para el analisis y procederemos a leer y mostrar el dataset.

In [1]:
import re
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from IPython.display import display

pd.set_option('display.max_columns', None)

In [3]:
url ='https://raw.githubusercontent.com/simonzanetti/2023.2-SysArmy-IT-Salaries-Survey/main/dataset.csv'
df = pd.read_csv(url)

display(df.head(1))

#print("Shape: " + str(df.shape))
#print("\nColumn Names and Types: ")
#print(df.dtypes)

,Estoy trabajando en,Dónde estás trabajando,Dedicación,Tipo de contrato,Último salario mensual o retiro BRUTO (en tu moneda local),Último salario mensual o retiro NETO (en tu moneda local),Pagos en dólares,Si tu sueldo está dolarizado ¿Cuál fue el último valor del dólar que tomaron?,Recibís algún tipo de bono,A qué está atado el bono,¿Tuviste actualizaciones de tus ingresos laborales durante 2023?,¿De qué % fue el ajuste total acumulado?,¿En qué mes fue el último ajuste?,Cómo considerás que están tus ingresos laborales comparados con el semestre anterior,¿Contás con beneficios adicionales?,¿Qué tan conforme estás con tus ingresos laborales?,Trabajo de,Años de experiencia,Antigüedad en la empresa actual,Tiempo en el puesto actual,¿Cuántas personas a cargo tenés?,Plataformas que utilizas en tu puesto actual,Lenguajes de programación o tecnologías que utilices en tu puesto actual,"Frameworks, herramientas y librerías que utilices en tu puesto actual",Bases de datos,QA / Testing,Cantidad de personas en tu organización,Modalidad de trabajo,Si trabajás bajo un esquema híbrido ¿Cuántos días a la semana vas a la oficina?,¿La recomendás como un buen lugar para trabajar?,"¿Qué tanto estás usando Copilot, ChatGPT u otras herramientas de IA para tu trabajo?",¿Salir o seguir contestando?,Máximo nivel de estudios,Estado,Carrera,Institución educativa,¿Salir o seguir contestando sobre las guardias?,¿Tenés guardias?,Cuánto cobrás por guardia,Aclará el número que ingresaste en el campo anterior,¿Salir o seguir contestando sobre estudios?,Tengo (edad),Me identifico (género)
0,Argentina,Catamarca,Full-Time,Staff (planta permanente),680000.0,520000.0,NaN,NaN,No,No recibo bono,Uno,26.0,Abril,2,"Abono de Internet, Clases de idiomas, Crédito ...",2,Infosec,10.0,1.0,1.0,0,"Amazon Web Services, Linux","Bash/Shell, Python",Angular,Ninguna de las anteriores,Ninguna de las anteriores,De 201 a 500 personas,100% remoto,0,5,0,Terminar encuesta,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36,Varón Cis
